In [97]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

Initial data cleaning

In [98]:
df = pd.read_csv("customerchurnds.csv")
df.drop(columns="customerID", inplace = True)
df = df[df["TotalCharges"] != " "]
df["TotalCharges"] = pd.to_numeric(df.TotalCharges)
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

Useful functions for next step

In [99]:
def finduniq(df):
    for column in df:
        print(f"{column} : {df[column].unique()}")

def findvals(df, valueset):
    yesandno = []
    for column in df:
        if set(df[column].unique()) == valueset:
            yesandno.append(column)
    return yesandno

Label encoding binary features

In [100]:
df.replace(['No internet service', 'No phone service'], "No", inplace = True)
df.replace({i : {"Yes" : 1, "No" : 0} for i in findvals(df, {"Yes", "No"})}, inplace=True)
df.replace({i : {"Male" : 1, "Female" : 0} for i in findvals(df, {"Male", "Female"})}, inplace=True)

One hot encoding

In [101]:
objectColumns = df.select_dtypes(include="object").columns
df = pd.get_dummies(data=df, columns=objectColumns, drop_first=True)
df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,MonthlyCharges,TotalCharges,Churn,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,0,1,0,0,0,1,0,...,29.85,29.85,0,0,0,0,0,0,1,0
1,1,0,0,0,34,1,0,1,0,1,...,56.95,1889.50,0,0,0,1,0,0,0,1
2,1,0,0,0,2,1,0,1,1,0,...,53.85,108.15,1,0,0,0,0,0,0,1
3,1,0,0,0,45,0,0,1,0,1,...,42.30,1840.75,0,0,0,1,0,0,0,0
4,0,0,0,0,2,1,0,0,0,0,...,70.70,151.65,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,1,1,24,1,1,1,0,1,...,84.80,1990.50,0,0,0,1,0,0,0,1
7039,0,0,1,1,72,1,1,0,1,1,...,103.20,7362.90,0,1,0,1,0,1,0,0
7040,0,0,1,1,11,0,0,1,0,0,...,29.60,346.45,0,0,0,0,0,0,1,0
7041,1,1,1,0,4,1,1,0,0,0,...,74.40,306.60,1,1,0,0,0,0,0,1


In [104]:
df.dtypes

gender                                     int64
SeniorCitizen                              int64
Partner                                    int64
Dependents                                 int64
tenure                                     int64
PhoneService                               int64
MultipleLines                              int64
OnlineSecurity                             int64
OnlineBackup                               int64
DeviceProtection                           int64
TechSupport                                int64
StreamingTV                                int64
StreamingMovies                            int64
PaperlessBilling                           int64
MonthlyCharges                           float64
TotalCharges                             float64
Churn                                      int64
InternetService_Fiber optic                uint8
InternetService_No                         uint8
Contract_One year                          uint8
Contract_Two year   

MinMax Scaling

In [106]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
for column in df:
    if column not in findvals(df, {0, 1}):
        df[column] = scaler.fit_transform(df[[column]])
df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,...,MonthlyCharges,TotalCharges,Churn,InternetService_Fiber optic,InternetService_No,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,0,1,0,0.000000,0,0,0,1,0,...,0.115423,0.001275,0,0,0,0,0,0,1,0
1,1,0,0,0,0.464789,1,0,1,0,1,...,0.385075,0.215867,0,0,0,1,0,0,0,1
2,1,0,0,0,0.014085,1,0,1,1,0,...,0.354229,0.010310,1,0,0,0,0,0,0,1
3,1,0,0,0,0.619718,0,0,1,0,1,...,0.239303,0.210241,0,0,0,1,0,0,0,0
4,0,0,0,0,0.014085,1,0,0,0,0,...,0.521891,0.015330,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,1,1,0.323944,1,1,1,0,1,...,0.662189,0.227521,0,0,0,1,0,0,0,1
7039,0,0,1,1,1.000000,1,1,0,1,1,...,0.845274,0.847461,0,1,0,1,0,1,0,0
7040,0,0,1,1,0.140845,0,0,1,0,0,...,0.112935,0.037809,0,0,0,0,0,0,1,0
7041,1,1,1,0,0.042254,1,1,0,0,0,...,0.558706,0.033210,1,1,0,0,0,0,0,1


--- Deep Learning Stage ---

In [117]:
from sklearn.model_selection import train_test_split

X = df.drop(columns="Churn")
y = df["Churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [181]:
model = keras.Sequential([
    keras.layers.Dense(23, input_shape=(23,), activation="relu"),
    keras.layers.Dense(46, activation="tanh"),
    keras.layers.Dense(23, activation="tanh"),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer="Adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

In [182]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
154/154 [==============================] - 0s 661us/step - loss: 0.4899 - accuracy: 0.7580
Epoch 2/100
154/154 [==============================] - 0s 674us/step - loss: 0.4334 - accuracy: 0.7877
Epoch 3/100
154/154 [==============================] - 0s 778us/step - loss: 0.4244 - accuracy: 0.7922
Epoch 4/100
154/154 [==============================] - 0s 733us/step - loss: 0.4224 - accuracy: 0.7948
Epoch 5/100
154/154 [==============================] - 0s 733us/step - loss: 0.4168 - accuracy: 0.7999
Epoch 6/100
154/154 [==============================] - 0s 811us/step - loss: 0.4137 - accuracy: 0.8019
Epoch 7/100
154/154 [==============================] - 0s 805us/step - loss: 0.4117 - accuracy: 0.8023
Epoch 8/100
154/154 [==============================] - 0s 792us/step - loss: 0.4094 - accuracy: 0.8019
Epoch 9/100
154/154 [==============================] - 0s 962us/step - loss: 0.4102 - accuracy: 0.8029
Epoch 10/100
154/154 [==============================] - 0s 739us/step - l

154/154 [==============================] - 0s 667us/step - loss: 0.3524 - accuracy: 0.8302
Epoch 80/100
154/154 [==============================] - 0s 844us/step - loss: 0.3531 - accuracy: 0.8322
Epoch 81/100
154/154 [==============================] - 0s 890us/step - loss: 0.3502 - accuracy: 0.8320
Epoch 82/100
154/154 [==============================] - 0s 870us/step - loss: 0.3500 - accuracy: 0.8322
Epoch 83/100
154/154 [==============================] - 0s 903us/step - loss: 0.3491 - accuracy: 0.8326
Epoch 84/100
154/154 [==============================] - 0s 844us/step - loss: 0.3488 - accuracy: 0.8379
Epoch 85/100
154/154 [==============================] - 0s 857us/step - loss: 0.3463 - accuracy: 0.8358
Epoch 86/100
154/154 [==============================] - 0s 837us/step - loss: 0.3465 - accuracy: 0.8346
Epoch 87/100
154/154 [==============================] - 0s 778us/step - loss: 0.3446 - accuracy: 0.8342
Epoch 88/100
154/154 [==============================] - 0s 765us/step - loss:

In [183]:
model.evaluate(X_test, y_test)

66/66 [==============================] - 0s 601us/step - loss: 0.4904 - accuracy: 0.7929


[0.49043580889701843, 0.7928909659385681]